In [ ]:
using Plots
using LaTeXStrings

# TAREA 3

# FÍSICA COMPUTACIONAL

## Aquino Chapa Martha Abril

Hecho en Julia 🔴🟢🟣

#### 1.- En clase vimos el método de Euler mejorado en el que nosotros introducimos el Jacobiano de forma analítica para ser evaluado por el método de integración. Realice una función que implemente el mismo método de integración, pero en lugar de calcular el Jacobiano con una función que lo evalúa analíticamente, ahora sea calculado de manera numérica utilizando la derivación numérica.

Iniciemos obteniendo la función que nos de el jacobiano. Para ello, tomemos en cuenta que la deerivada parcial de una función será aumentando en la variable respecto a la cual derivaremos, el aumento:
$$\frac{\partial f(\vec{x}_0)}{\partial x_i}=\frac{f(\vec{x}_0+h\vec{e_i})-f(f(\vec{x}_0-h\vec{e_i})}{2h}$$

Y debido a que en el jacobiano requiremos la derivada parcial de la j-ésima entrada del jacobiano, a partir de lo anterior esta estará dada por:
$$\frac{\partial f(\vec{x}_0)_j}{\partial x_i}=\frac{f(\vec{x}_0+h\vec{e_i})_j-f(f(\vec{x}_0-h\vec{e_i})_j}{2h}$$
Dicho lo anterior, definamos la función que nos de la matriz jacobiana.

In [4]:
#jacobiano autónomo 
#la entrada xi consiste en las variables espaciales del problema, donde obtendremos las parciales
#h el paso para la obtención de las derivadas parciales
#func es la función a la cual sacaremos el jacobiano
#t el tiempo donde evaluaremos la parcial temporal
function jacobianotemp(xi,h,func,t)
    #Definimos las entradas de la matriz
    jac=zeros(length(func(xi,t)),length(xi)+1)
    #hacemos un ciclo donde iremos llenando las entradas de nuestra matriz.
    for i in 1:length(func(xi,t)), j in 1:length(xi)
        #Definiremlos los vetores xp1 y xp2, como los vectores que tiene un aumento o desaumento en la i-ésima entrda.
        xp1=zeros(length(xi))
        xp2=zeros(length(xi))
        #Creamos un ciclo con el cual definiremos las entradas de los vectores donde evaluaremos la función para obtener la derivada parcial
        for k in 1:length(xi)
            #Aqui sólo agregamos el incremento a la j-ésima entrada del vector
            if  k==j 
                xp1[k]=xi[j]+h
                xp2[k]=xi[j]-h
            else
                #Dejamos todas las demás entradas igual
                xp1[k]=xi[j]
                xp2[k]=xi[j]
            end
        end
        #Definimos la entrada ij de la matriz jacobiana, sin tomar en cuenta la entrada de la parcial temporal
        jac[i,j]=(func(xp1,t)[i]-func(xp2,t)[i])/(xp1[j]-xp2[j])
        #definimos las componentes de la columna de la parcial temporal
        jac[i,length(xi)+1]=(func(xi,t+h)[i]-func(xi,t-h)[i])/(2h)
    end
    return jac
end

jacobianotemp (generic function with 1 method)

Ahora modifiquemos la función Euler_mejorado_J_t que hace uso del jacobiano con dependencia temporal para anexarle el Jacobiano analítico.

In [5]:
#Debemos nuestra funcion euler mejorado consistirá de la función que nos da la ecuación diferencia (edo)
#La función edo_1, que es la función, con un elemento 1 en la última entrada, ejemplo [x,y,z,1]
#p_ini las condiciones iniciales de la posición
#t el tiempo
function Euler_mejorado_Jnum(edo,edo_1,p_ini,t)
    # J y g con dependencia temporal.
    sol = zeros(length(t),length(p_ini))
    sol[1,:] = p_ini
    δ = t[2]-t[1]
    for i in 1:(length(t)-1)
        eval_edo = edo(sol[i,:],t[i])
        #definimos g* que es la función por la que multiplicaremos el jacobiano
        g=edo_1(sol[i,:],t[i])
        if length(eval_edo) == length(p_ini)
            sol[i+1,:] .= sol[i,:] .+ δ .*eval_edo
        else
            sol[i+1,:] .= sol[i,:] .+ δ .*eval_edo[1:(end-1)]
        end
        #cambiamos jacob por nuestra función del jacobiano que obtuvimos
        sol[i+1,:] .+= 0.5*(δ^2) .*(jacobianotemp(sol[i,:],δ,edo,t[i]) *g )
    end
    return sol
end

Euler_mejorado_Jnum (generic function with 1 method)

Probemos que nuestra función sirve tomando el sistema de ecuaciones diferenciales del ejercicio 2 (se puede analizar con más detenimiento la obteniención del sistema de ecuaciones en la sección del ejercicio 2).
Iniciamos definiendo la función que nos de el sistema de ecuaciones diferenciales:

In [6]:
function sistema1_t(xyz,t)
    # xyz = [x1, x2, x3]
    dx = xyz[2]
    dy = xyz[3]
    dz = -xyz[3]*xyz[3] + 3*xyz[2]*xyz[2]*xyz[2] - cos(xyz[1])*cos(xyz[1]) + exp(-1*t)*sin(3*t)
    return [dx,dy,dz]
end

sistema1_t (generic function with 1 method)

Definimos una función auxilia que sea similar a la función anterior pero expandida, con una segunda entrada unitaria, que es la que nos dará g*.

In [7]:
function sistema1_ta(xyz,t)
    # xyz = [x1, x2, x3]
    dx = xyz[2]
    dy = xyz[3]
    dz = -xyz[3]*xyz[3] + 3*xyz[2]*xyz[2]*xyz[2] - cos(xyz[1])*cos(xyz[1]) + exp(-1*t)*sin(3*t)
    return [dx,dy,dz,1.0]
end

sistema1_ta (generic function with 1 method)

In [8]:
#Definimos el tiempo donde evaluaremos:
tiempo = collect(1:0.1:1.5)
solucion = Euler_mejorado_Jnum(sistema1_t,sistema1_ta,[1.0,2.0,1.0],tiempo)

6×3 Array{Float64,2}:
 1.0      2.0       1.0
 1.205    2.2138    3.23192
 1.44254  2.64661   5.43167
 1.73436  3.31959   8.33056
 2.10797  4.35329  13.1254
 2.60893  6.03956  21.9648

Apreciamos que funciona la función y no modifica las condiciones iniciales.

### 2.- Utilizando los integradores de Euler mejorado con Jacobiano analítico, Euler mejorado con Jacobiano numérico, Runge-Kutta de $2^{\circ}$ orden y Runge-Kutta de $4^{\circ}$ orden, encuentre un paso de integración que proporcione una precisión de $10^{−4}$ en la solución para la siguiente ecuación:

$$
\dddot{y} + \ddot{y}^2 - 3\dot{y}^3 + \cos^2{y} = e^{-t}\sin{3t}
$$

##### Con condiciones iniciales $\ddot{y}(1)=1$, $\dot{y}(1)=2$, $y(1)=1$. En particular, ¿qué sucede en en $t\in[1,2]$ y $t\in[1,2.1]$?

Extraemos las funciones vistas en clase, iniciamos en el Euler_mejorado con dependencia temporal: 

In [21]:
function Euler_mejorado_J_t(edo,p_ini,t,jacob)
    # J y g con dependencia temporal.
    sol = zeros(length(t),length(p_ini))
    sol[1,:] = p_ini
    δ = t[2]-t[1]
    for i in 1:(length(t)-1)
        eval_edo = edo(sol[i,:],t[i])
        if length(eval_edo) == length(p_ini)
            sol[i+1,:] .= sol[i,:] .+ δ .*eval_edo
        else
            sol[i+1,:] .= sol[i,:] .+ δ .*eval_edo[1:(end-1)]
        end
        sol[i+1,:] .+= 0.5*(δ^2) .*(jacob(sol[i,:],t[i]) *eval_edo )
    end
    return sol
end

Euler_mejorado_J_t (generic function with 1 method)

Runge-Kutta de 2do orden: 

In [10]:
function RK_2(edo,x_ini,t)
    sol = zeros( length(t) , length(x_ini) )
    sol[1,:] .= x_ini
    δ = t[2]-t[1]
    for i in 1:length(t)-1
        k1 = sol[i,:] .+ 0.5*δ .*edo(sol[i,:],t[i])
        sol[i+1,:] .= sol[i,:] .+ δ*edo(k1,t[i]+0.5*δ)
    end
    return sol
end

RK_2 (generic function with 1 method)

Runge-Kutta de 4to orden:

In [11]:
function RK_4(edo,x_ini,t)
    sol = zeros( length(t) , length(x_ini) )
    sol[1,:] .= x_ini
    δ = t[2]-t[1]
    for i in 1:length(t)-1
        k1 = edo(sol[i,:],t[i])
        k2 = edo(sol[i,:] .+ 0.5*δ.*k1 , t[i] + 0.5*δ)
        k3 = edo(sol[i,:] .+ 0.5*δ.*k2 , t[i] + 0.5*δ)
        k4 = edo(sol[i,:] .+ δ.*k2 , t[i] + δ)
        sol[i+1,:] .= sol[i,:] .+ (δ/6.0).*(k1 .+ 2.0.*k2 .+ 2.0.*k3 .+ k4)
    end
    return sol
end

RK_4 (generic function with 1 method)

Ahora definimos las funciones de las ecuaciones diferenciales. Para ello notemos que podemos reescribir la ecuación diferencial como un sistema de ecuaciones diferenciales:

$$
x_1=y\\
\dot{x}_1 = x_2\\
\dot{x}_2 = x_3\\
\dot{x}_3 = -x_3^2 + 3x_2^3 - \cos^2{(x_1)} + e^{-t}\sin{(3t)}
$$

Y cuyas condiciones iniciales son: $x_1(1)=1$, $x_2(1)=2$ y $x_3(1)=1$. Las cuales, podemos escribir en una función de la siguiente manera:

Notemos entonces que el jacobiano del sistema es:
$$J^{*}=\begin{pmatrix}
0 & 1 & 0&0\\
0 & 0 & 1&0\\
2\sin(x_1)\cos(x_1) & 9x_2^{2} & -2x_3&3e^{-t}\cos(3t)-e^{-t}\sin(3t)
\end{pmatrix}$$

Definimos la función que nos de la ecuación diferencial:

In [16]:
function EDO1(vec,t)
    # vec = [x1, x2, x3]
    dx1 = vec[2]
    dx2 = vec[3]
    dx3 = -vec[3]*vec[3] + 3*vec[2]*vec[2]*vec[2] - cos(vec[1])*cos(vec[1]) + exp(-t)*sin(3*t)
    return [dx1,dx2,dx3]
end

EDO1 (generic function with 1 method)

Definimos la función para la ecuación diferencial, aumentandole 1 en la última entrada.

In [17]:
function EDO1a(vec,t)
    # vec = [x1, x2, x3]
    dx1 = vec[2]
    dx2 = vec[3]
    dx3 = -vec[3]*vec[3] + 3*vec[2]*vec[2]*vec[2] - cos(vec[1])*cos(vec[1]) + exp(-t)*sin(3*t)
    return [dx1,dx2,dx3,1.0]
end

EDO1a (generic function with 1 method)

Y ahora definimos su Jacobiano analítico

In [18]:
function EDO1_J(vec,t)
    # vec = [x1, x2, x3]
    J = zeros(3,4)
    J[1,2] = 1.0
    J[2,3] = 1.0
    J[3,1] = 2*cos(vec[1])*sin(vec[1])
    J[3,2] = 9*vec[2]*vec[2]
    J[3,3] = -2*vec[3]
    J[3,4] = 3*exp(-1*t)*cos(3*t)-exp(-1*t)*sin(3*t)
    return J
end

EDO1_J (generic function with 1 method)

Ahora resolvamos esta ecuación diferencial con los diversos métodos en t=[1,2], para obtener una convergencia menor a $10^{-4}$ escojamos un paso que nos de minímo 4 cifras significativas. 

Iniciemos con el método de euler con jacobiano analítico.

In [45]:
tEulerAnalitico = collect(1:0.001:1.8)
solEulerAnalitico= Euler_mejorado_J_t(EDO1a,[1.0,2.0,1.0],tEulerAnalitico,EDO1_J)

801×3 Array{Float64,2}:
 1.0       2.0        1.0
 1.002     2.00101    1.02276
 1.004     2.00205    1.0455
 1.006     2.0031     1.06824
 1.00801   2.00418    1.09097
 1.01001   2.00528    1.11369
 1.01202   2.00641    1.1364
 1.01403   2.00756    1.15909
 1.01603   2.00873    1.18178
 1.01804   2.00992    1.20446
 1.02005   2.01114    1.22713
 1.02207   2.01237    1.24979
 1.02408   2.01364    1.27244
 ⋮                  
 6.99906  37.0254   382.449
 7.03627  37.4109   388.516
 7.07388  37.8025   394.713
 7.11188  38.2003   401.042
 7.15028  38.6046   407.508
 7.18909  39.0153   414.113
 7.22831  39.4328   420.861
 7.26795  39.857    427.757
 7.30803  40.2883   434.805
 7.34853  40.7267   442.008
 7.38948  41.1723   449.372
 7.43088  41.6254   456.899

Notemos que si tomamos t en [1,1.8], las soluciones se encuentran bien definidas, sin embargo, si cambiamos este intervalo a [1,1.9], [1,2] o [1,2.1] las soluciones no se encuentran bien defidas:

In [49]:
tEulerAnaliticoD = collect(1:0.001:2.1)
solEulerAnaliticoD= Euler_mejorado_J_t(EDO1a,[1.0,2.0,1.0],tEulerAnalitico,EDO1_J)

1101×3 Array{Float64,2}:
   1.0        2.0        1.0
   1.002      2.00101    1.02276
   1.004      2.00205    1.0455
   1.006      2.0031     1.06824
   1.00801    2.00418    1.09097
   1.01001    2.00528    1.11369
   1.01202    2.00641    1.1364
   1.01403    2.00756    1.15909
   1.01603    2.00873    1.18178
   1.01804    2.00992    1.20446
   1.02005    2.01114    1.22713
   1.02207    2.01237    1.24979
   1.02408    2.01364    1.27244
   ⋮                   
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN
 NaN        NaN        NaN

Veamos ahora con el jacobiano númerico en t[1,1.8]

In [48]:
tEulerNum = collect(1:0.00001:1.8)
solEulerAnalitico= Euler_mejorado_Jnum(EDO1,EDO1a,[1.0,2.0,1.0],tEulerNum)

80001×3 Array{Float64,2}:
 1.0       2.0        1.0
 1.00002   2.00001    1.00023
 1.00004   2.00002    1.00046
 1.00006   2.00003    1.00068
 1.00008   2.00004    1.00091
 1.0001    2.00005    1.00114
 1.00012   2.00006    1.00137
 1.00014   2.00007    1.00159
 1.00016   2.00008    1.00182
 1.00018   2.00009    1.00205
 1.0002    2.0001     1.00228
 1.00022   2.00011    1.0025
 1.00024   2.00012    1.00273
 ⋮                  
 7.42659  41.5793   456.131
 7.42701  41.5838   456.207
 7.42742  41.5884   456.283
 7.42784  41.593    456.359
 7.42825  41.5975   456.435
 7.42867  41.6021   456.511
 7.42908  41.6066   456.587
 7.4295   41.6112   456.663
 7.42992  41.6158   456.74
 7.43033  41.6203   456.816
 7.43075  41.6249   456.892
 7.43117  41.6295   456.968

Con el método de Runge-Kutta de 2do orden en t[1,1.8]:

In [55]:
tRK2 = collect(1:0.0001:1.8)
solRK2 = RK_2(EDO1,[1.0,2.0,1.0],tiempo)

501×3 Array{Float64,2}:
 1.0      2.0       1.0
 1.002    2.00101   1.02276
 1.004    2.00205   1.0455
 1.006    2.0031    1.06824
 1.00801  2.00418   1.09097
 1.01001  2.00528   1.11369
 1.01202  2.00641   1.13639
 1.01403  2.00756   1.15909
 1.01603  2.00873   1.18178
 1.01804  2.00992   1.20446
 1.02005  2.01114   1.22713
 1.02207  2.01237   1.24979
 1.02408  2.01364   1.27244
 ⋮                 
 2.58926  6.00528  22.6139
 2.59528  6.02796  22.7518
 2.60132  6.05078  22.8909
 2.60738  6.07374  23.0311
 2.61346  6.09684  23.1725
 2.61957  6.12009  23.315
 2.6257   6.14347  23.4587
 2.63186  6.167    23.6037
 2.63804  6.19068  23.7498
 2.64424  6.2145   23.8971
 2.65047  6.23848  24.0457
 2.65672  6.2626   24.1955

Con el método de Runge-Kutta de 4do orden en t[1,1.8]:

In [56]:
tRK4 = collect(1:0.0001:1.8)
solRK4 = RK_4(EDO1,[1.0,2.0,1.0],tiempo)

501×3 Array{Float64,2}:
 1.0      2.0       1.0
 1.002    2.00101   1.02276
 1.004    2.00205   1.0455
 1.006    2.0031    1.06824
 1.00801  2.00418   1.09097
 1.01001  2.00528   1.11369
 1.01202  2.00641   1.1364
 1.01403  2.00756   1.15909
 1.01603  2.00873   1.18178
 1.01804  2.00992   1.20446
 1.02005  2.01114   1.22713
 1.02207  2.01237   1.24979
 1.02408  2.01364   1.27244
 ⋮                 
 2.58927  6.00532  22.6141
 2.59528  6.028    22.752
 2.60132  6.05082  22.8911
 2.60739  6.07378  23.0313
 2.61347  6.09689  23.1727
 2.61958  6.12013  23.3153
 2.62571  6.14352  23.459
 2.63187  6.16705  23.6039
 2.63805  6.19073  23.75
 2.64425  6.21455  23.8974
 2.65048  6.23852  24.0459
 2.65673  6.26264  24.1958

#### 3.- Modifique las funciones integradoras que programamos en clase (Euler mejorado, Runge-Kutta de $2^{\circ}$ orden y Runge-Kutta de $4^{\circ}$ orden) para que se puedan pasar argumentos opcionales al sistema de ecuaciones diferenciales de tal manera que se puedan modificar parámetros al momento de hacer la integración. Una vez hecho esto, obtenga la solución de la siguiente ecuación diferencial

$$
\ddot{x} + \frac{1}{10}\dot{x}+4\sin{x} = \frac{1}{2}\sin{(\omega t)}
$$

#### Con condiciones iniciales $\dot{x}(0)=1$ y $x(0)=1$ y para $\omega\in\{1,1.1,1.2,1.3,...,2.9,3\}$. Realice la gráfica de algunas soluciones que representen el comportamiento de la solución en función de $\omega$ y explique su resultado. La solución debe tener una precisión de por lo menos $10^{−5}$.

Notemos que podemos escribir la ecuación diferencial como un sistema de ecuaciones diferenciales donde:
$$
x=x_1\\
\dot{x}_1 = x_2\\
\dot{x}_2 = -\frac{1}{10}x_2 - 4\sin{(x_1)} + \frac{1}{2}\sin{(\omega t)}
$$
Ahora escribamos este sistema como una función en Julia y su jacobiano de manera analítica.

In [ ]:
function EDO2(vec,t,ω)
    # vec = [x1, x2]
    dx1 = vec[2]
    dx2 = -vec[2]/10 - 4*sin(vec[1]) + 0.5*sin(ω*t)
    return [dx1,dx2,1.0]
end

Ahora el jacobiano analítico:

In [ ]:
function EDO2_J(vec,t,ω)
    # param = [ omega ]
    # vec = [x1, x2]
    #ω = param
    J = zeros(length(vec),length(vec)+1)
    J[1,2] = 1.0
    J[2,1] = -4*cos(vec[1])
    J[2,2] = -1/10
    J[2,3] = 0.5*ω*cos(ω*t)
    return J
end

Ahora modifiquemos las funciones de Euler mejorado:

In [ ]:
function Euler_mejorado_omega(edo,p_ini,t,jacob,ω)
    # J y g con dependencia temporal y de parametros externos
    sol = zeros(length(t),length(p_ini))
    sol[1,:] = p_ini
    δ = t[2]-t[1]
    for i in 1:(length(t)-1)
        eval_edo = edo(sol[i,:],t[i],ω)
        if length(eval_edo) == length(p_ini)
            sol[i+1,:] .= sol[i,:] .+ δ .*eval_edo
        else
            sol[i+1,:] .= sol[i,:] .+ δ .*eval_edo[1:(end-1)]
        end
        sol[i+1,:] .+= 0.5*(δ^2) .*(jacob(sol[i,:],t[i],ω) *eval_edo )
    end
    return sol
end

Definimos la ecuación diferencia para los métodos de Runge-Kutta.

In [ ]:
function EDO2_1(vec,t,ω)
    # vec = [x1, x2]
    dx1 = vec[2]
    dx2 = -vec[2]/10 - 4*sin(vec[1]) + 0.5*sin(ω*t)
    return [dx1,dx2]#,1.0]
end

Modificamos la función de Runge-Kutta de 2do orden:

In [ ]:
function RK_2_1(edo,x_ini,t,ω)
    sol = zeros( length(t) , length(x_ini) )
    sol[1,:] .= x_ini
    δ = t[2]-t[1]
    for i in 1:length(t)-1
        k1 = sol[i,:] .+ 0.5*δ .*edo(sol[i,:],t[i],ω) #solo agregamos un param para que no haya problemas de matcheo
        sol[i+1,:] .= sol[i,:] .+ δ*edo(k1,t[i]+0.5*δ,ω)
    end
    return sol
end

Modificamos la función de Runge-Kutta de 4to orden:

In [ ]:
function RK_4_1(edo,x_ini,t,ω)
    sol = zeros( length(t) , length(x_ini) )
    sol[1,:] .= x_ini
    δ = t[2]-t[1]
    for i in 1:length(t)-1
        k1 = edo(sol[i,:],t[i],ω)
        k2 = edo(sol[i,:] .+ 0.5*δ.*k1 , t[i] + 0.5*δ,ω)
        k3 = edo(sol[i,:] .+ 0.5*δ.*k2 , t[i] + 0.5*δ,ω)
        k4 = edo(sol[i,:] .+ δ.*k2 , t[i] + δ,ω)
        sol[i+1,:] .= sol[i,:] .+ (δ/6.0).*(k1 .+ 2.0.*k2 .+ 2.0.*k3 .+ k4)
    end
    return sol
end

In [ ]:
plot(title="Sistema con diferentes valores para omega usando Euler Mejorado",xlabel="t",ylabel="x(t)")
tiempo = collect(0:0.01:70π)
omegas = collect(1:0.5:3) #tomamos solamente 5 valores de ω
for ω in omegas
    solucion = RK_4_1(EDO2_1,[1.0,0.0],tiempo,ω)
    plot!(tiempo,solucion[:,1],label ="omega=$(ω)", lw=1.0)
end
plot!()

Ya que probamos nuestras funciones, entonces resolvamos el sistema con las condiciones dadas y para diversas omegas, para ello, encontremos el espacio en nuestra malla para que nos de un error menor a 10^{-5}

####  4.- Considere el siguiente sistema de ecuaciones diferenciales

$$
\frac{dx}{dt} = 10(y-x)\\
\frac{dy}{dt} = x(28-z) - y\\
\frac{dz}{dt} = xy - \frac{8}{3}z
$$

#### Realice una integración numérica del sistema de ecuaciones con el método de Runge-Kutta de $4^{\circ}$ y realice los siguientes ejercicios:
#### (a)  Elija una condición inicial aleatoria en el $x,y,z\in[-5,5]$ y realice la gráfica de la solución en el plano $xy$, $xz$ y $yz$, comente sus observaciones. 

#### (b) Ahora tome 3 soluciones parecidas. Para ello proponga 3 condiciones iniciales tales que en la coordenada en $y$ las condiciones iniciales difieran entre si en $10^{−2}$.

#### Nota: De preferencia realice integraciones con tiempos relativamente largos en los que pueda apreciar diferencias en el comportamiento de las soluciones. Además asegure que cada una de las soluciones que encuentre sea convergente en por lo menos 5 cifras ($10^{-5}$), es decir encuentre un paso adecuado para realizar la integración.

Escribamos la función que nos de el sistema de ecuaciones:

In [ ]:
function EDO3(vec,t)
    # vec = [x1, x2, x3]
    dx = 10*vec[2] - 10*vec[1]
    dy = 28*vec[1] - vec[1]*vec[3] - vec[2]
    dz = vec[1]*vec[2] - 8*vec[3]/3
    return [dx,dy,dz]
end

Para escojer nuestras condiciones iniciales tomemos 5 y un valor aleatorio entre 0 y 1.

In [ ]:
vec0=[5rand(),-5rand(),5rand()]

Veamos cual es el valor del paso para tener la condición de la convergencia pedida.

In [ ]:
function convergencia(h)
    tiempo = collect(0:h:13π)
    sol1=RK_4(EDO3,vec0,tiempo) #introducimos las condiciones iniciales aleatorias
    sol2=RK_2(EDO3,vec0,tiempo)
    #error en x
    m=length(tiempo)
    ex=collect(zeros(length(m)))
    ey=collect(zeros(length(sol1[:,2])))
    ez=collect(zeros(length(sol1[:,3])))
    for i in 1:length(sol1[:,1])
        ex[i]= (abs(sol1[i,1]-sol2[i,1]))
        ey[i]= (abs(sol1[i,2]-sol2[i,2]))
        ez[i]= (abs(sol1[i,3]-sol2[i,3]))
    end
    m1=max(ex)
    m2=max(ey)
    m3=max(ez)
    return maximum(m1,m2,m3)
    
end

In [ ]:
convergencia(10)

Realicemos la gráfica de la solución en el plano xy:

In [ ]:
tiempo = collect(0:0.01:13π)
solucion4 = RK_4(EDO3,vec0,tiempo) #introducimos las condiciones iniciales aleatorias
plot(title="Espacio Fase en el plano xy",xlabel="x(t)",ylabel="y(t)")
plot!(solucion4[:,1],solucion4[:,2])
plot!(legend=false)

Para el plano xz:

In [ ]:
plot(title="Espacio Fase en el plano xz",xlabel="x(t)",ylabel="z(t)")
plot!(solucion4[:,1],solucion4[:,3])
plot!(legend=false)

En el plano yx:

In [ ]:
plot(title="Espacio Fase en el plano yz",xlabel="y(t)",ylabel="z(t)")
plot!(solucion4[:,2],solucion4[:,3])
plot!(legend=false)

Observaciones:

b) Iniciamos escogiendo nuestras soluciones en función de las condiciones inciales pasadas.

In [ ]:
vec0_1=[vec0[1]+0.1,vec0[2]+0.001,vec0[3]-0.1]
tiempo = collect(0:0.01:13π)
solucion4_1 = RK_4(EDO3,vec0_1,tiempo)

Obtengamos los espacio fase para la solución con la segunda solución:

In [ ]:
plot(title="Espacio Fase en el plano xy",xlabel="x(t)",ylabel="z(t)")
plot!(solucion4_1[:,1],solucion4_1[:,2])
plot!(legend=false)

In [ ]:
plot(title="Espacio Fase en el plano xz",xlabel="x(t)",ylabel="z(t)")
plot!(solucion4_1[:,1],solucion4_1[:,3])
plot!(legend=false)

In [ ]:
plot(title="Espacio Fase en el plano yz",xlabel="x(t)",ylabel="z(t)")
plot!(solucion4_1[:,2],solucion4_1[:,3])
plot!(legend=false)

Chequemos la precisión de nuestros calculos obteniendo la diferencia entre la solución dada por el código Rugen-Kutta de orden 2 con el de 4to orden.

In [ ]:
presicion(0.1)